In [1]:
import pandas as pd
UB = pd.read_csv('tailard_UB.csv')
UB = pd.DataFrame(UB)

In [2]:
from urllib.request import Request, urlopen
import urllib.parse
from bs4 import BeautifulSoup
import numpy as np
import requests

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}

In [3]:
opt_list = dict()
dur_list = dict()
mch_list = dict() 

In [4]:
index = list(range(0, 80))
index.remove(61)

In [5]:
for i in index:
    size_text = UB.iloc[i, 1]
    size_list = size_text.rsplit('x', 1)
    n_jobs = int(size_list[0])
    n_mchs = int(size_list[1])

    UB_temp = UB.iloc[i, 2]

    i_2d = str(i+1).zfill(2)
    url = 'https://optimizizer.com/solution.php?name=ta' + str(i_2d) + '&UB=' + str(UB_temp) + '&problemclass=ta' 

    response = requests.get(url, headers=headers)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    nums = soup.get_text().rsplit(':', 1)[-1].rsplit(' back', 1)[0]
    table = list(map(int, nums.split(' ')))

    table_opt = np.array(table).reshape(n_mchs, n_jobs)
    opt_list[i+1] = table_opt

    # target_url = 'http://mistic.heig-vd.ch/taillard/problemes.dir/ordonnancement.dir/jobshop.dir/tai15_15.txt'
    # response = requests.get(target_url)
    # data = response.text
    # soup2 = BeautifulSoup(data, 'html.parser')

    # numbs = soup2.get_text().rsplit('Times')[3].rsplit('Machines')[2]
    # print(numbs)

    # # numbs = soup2.get_text().rsplit('Times')[1].rsplit('Machines')[0]
    # # new_mch = numbs.replace('\r\n', '')[1:]
    
    # # table2 = list(map(int, new_mch.split()))

    # # table_dur = np.array(table2).reshape(n_jobs, n_mchs)
    # # print(table_dur)

The ith row of each solution file represents the processing order of jobs on the ith machine
(indexing of jobs starts from zero)

In [7]:
n_pair = [(15, 15), (20, 15), (20, 20), (30, 15), (30, 20), (50, 15), (50, 20), (100, 20)]
index = 1

for (n_job, n_mch) in n_pair:
    for iter in range(10):
        with open('tai_' + str(n_job) + '_' + str(n_mch) + '.txt') as f:
            contents = f.read()
        lis = contents.rsplit('=')[iter*2].replace('\n', '').split()
        res = [eval(i) for i in lis]
        table2 = np.array(res).reshape(n_job, n_mch)
        dur_list[index] = table2

        with open('tai_' + str(n_job) + '_' + str(n_mch) + '.txt') as f:
            contents = f.read()
        lis2 = contents.rsplit('=')[iter*2+1].replace('\n', '').split()
        res2 = [eval(i) for i in lis2]
        table3 = np.array(res2).reshape(n_job, n_mch)
        mch_list[index] = table3

        print(index)
        index = index + 1


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80


In [8]:
opt_list[62]

KeyError: 62

In [34]:
import itertools as it

n_pair = [(15, 15), (20, 15), (20, 20), (30, 15), (30, 20), (50, 15), (50, 20), (100, 20)]
idx = 1
graph_dict = dict()


for (n_job, n_mch) in n_pair:
    for iter in range(10):
        if idx != 62:
            edge_list = []
            for mch_idx in range(n_mch):
                seq_temp = []
                for seq_idx in range(n_job):
                    i = opt_list[idx][mch_idx, seq_idx] #7
                    j = list(mch_list[idx][i]).index(mch_idx+1) #2
                    seq_temp.append((i)*n_mch + j)
                for (i, j) in list(it.pairwise(seq_temp)):
                    edge_list.append([i, j])

            for num in range(0, n_job*n_mch):
                if (num+1) % n_mch != 0:
                    edge_list.append([num, num+1])
        graph_dict[idx] = edge_list
        idx = idx + 1

             

In [63]:
import gurobipy as gp
from gurobipy import GRB

n_pair = [(15, 15), (20, 15), (20, 20), (30, 15), (30, 20), (50, 15), (50, 20), (100, 20)]
idx = 1

index = list(range(1, 81))
index.remove(62)

completion_dict = dict()


for i in index:

    n_job = n_pair[(i-1)//10][0]
    n_mch = n_pair[(i-1)//10][1]

    # Create a new model
    m = gp.Model("mip1")

    # Create variables
    var = list()
    for inst in range(n_job * n_mch):
        var.append(m.addVar(vtype=GRB.INTEGER, name = "x"+str(inst)))
    max_var = m.addVar(vtype=GRB.INTEGER, name = "max_var")

    # Add constraint: x + y >= 1
    for inst in range(n_job * n_mch):
        m.addConstr(var[inst] >= dur_list[i][inst // n_mch, inst % n_mch])

    for (u, v) in graph_dict[i]:
        m.addConstr(var[u] + dur_list[i][u // n_mch, u %  n_mch] <= var[v])

    for inst in range(len(var)):
        m.addConstr(var[inst] <= max_var, "c_max")

    # Set objective
    m.setObjective(max_var, GRB.MINIMIZE)

    # Optimize model
    m.optimize()
    m.Params.LogToConsole = 0

    completion_time = list()
    for v in m.getVars():
        completion_time.append(v.x)

    completion_dict[i] = np.array(completion_time[0:len(completion_time)-1]).reshape(n_job, n_mch)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: AMD Ryzen 7 2700X Eight-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 870 rows, 226 columns and 1515 nonzeros
Model fingerprint: 0x7c80b352
Variable types: 0 continuous, 226 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 870 rows and 226 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 1283 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.283000000000e+03, best bound 1.283000000000e+03, gap 0.0000%
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: AMD Ryzen 7 2700X Eight-Core Proc

In [65]:
completion_dict[62]

KeyError: 62

In [66]:

n_pair = [(15, 15), (20, 15), (20, 20), (30, 15), (30, 20), (50, 15), (50, 20), (100, 20)]
idx = 1

index = list(range(1, 81))
index.remove(62)

num_of_jobs = dict()

for i in index:
    n_job = n_pair[(i-1)//10][0]
    n_mch = n_pair[(i-1)//10][1]

    num_of_jobs[i] = np.array([list(range(0, n_mch)) for i in range(n_job)]).flatten()

In [72]:
index = list(range(1, 81))
index.remove(62)

final_temp = dict()
for i in index:
     result = np.transpose(np.array([completion_dict[i].flatten(), dur_list[i].flatten(), num_of_jobs[i]]))
     final_temp[i] = result

In [73]:
import torch
from torch_geometric.data import Data

index = list(range(1, 81))
index.remove(62)

final_result = dict()
for i in index:
    edge_index = torch.tensor(graph_dict[i], dtype = torch.long)
    x = torch.tensor(final_temp[i], dtype = torch.float)
    data = Data(x=x, edge_index = edge_index.t().contiguous())
    final_result[i] = data

In [75]:
final_result[1]

Data(x=[225, 3], edge_index=[2, 420])

In [76]:
n_pair = [(15, 15), (20, 15), (20, 20), (30, 15), (30, 20), (50, 15), (50, 20), (100, 20)]

index = list(range(1, 81))
index.remove(62)


for i in index:

    n_job = n_pair[(i-1)//10][0]
    n_mch = n_pair[(i-1)//10][1]

    torch.save(final_result[i], 'jssp_opt\jssp_'+str(n_job)+'_'+str(n_mch)+'_'+str((i-1)%10)+'.pt')